In [22]:
import pandas as pd
# Load data from QNA csv file
df = pd.read_csv("/kaggle/input/qnanswer/renewables.csv", usecols=['Paragraph','Question','Answer'])
pd.options.display.max_colwidth = 300
print(df.shape)
df.head(3)

(3354, 3)


,Paragraph,Question,Answer
0,"Figure 1. Trend in allocation of total NIH budget to NIH-designated Research-Project Grants (RPGs). Total NIH spending on RPGs (P01, P42, PN1, U01, U19, UC1, NIGMS P41, and all R-series) in each Federal Fiscal Year (FY) divided by total NIH budget. Analysis courtesy of FASEB, 2009, drawing on d...",What percentage of the Total NIH Budget is allocated to NIH-Designated Research-Project Grants (RPGs)?,"The percentage of the Total NIH Budget allocated to NIH-Designated Research-Project Grants (RPGs) varies by federal fiscal year (FY), but ranged from 27.0-27.7% between FY 2004-2008."
1,"Figure 1. Trend in allocation of total NIH budget to NIH-designated Research-Project Grants (RPGs). Total NIH spending on RPGs (P01, P42, PN1, U01, U19, UC1, NIGMS P41, and all R-series) in each Federal Fiscal Year (FY) divided by total NIH budget. Analysis courtesy of FASEB, 2009, drawing on d...",How much of the Total NIH Budget was allocated to R01/R01-Equivalent grants during the post-doubling period?,The percentage of the Total NIH Budget allocated to R01/R01-Equivalent grants during the post-doubling period was approximately 17.2%.
2,"A few data snapshots bear particular comment. Damage continues to worsen as the NIH appropriation loses ground to inflation year after year in the ""post-doubling era""\n \n3\n. However, it is equally true that the NIH decreased its allocation of funds to the gold standard of inves...",What is the full post-doubling era?,The full post-doubling era is the period between FY2003 and FY2008 where the NIH budget increased by $2.8 billion.


In [2]:
# equired libraries to install
!pip install transformers[torch]
!pip install rouge
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# PyTorch T5 model

In [15]:
# Load pretrained model
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_name = "t5-base"
model1 = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name, model_max_length=1024)

In [22]:
# create a function to get answers from your questions
def get_answer(question, context):
    input_text = "question: {} context: {}".format(question, context)
    input_ids = tokenizer.encode("{}".format(input_text), return_tensors="pt", 
                              max_length=512, truncation=True)
    # Generate the answer using T5 model
    output = model1.generate(input_ids,max_new_tokens=120)
    # Decode the generated answer
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [27]:
# Test
idx = 3
print(df.loc[idx, 'Question'])
print(df.loc[idx, 'Answer'])
print(get_answer(df.loc[idx, 'Question'], df.loc[idx, 'Paragraph']))

How did the NIH budget increase during the post-doubling era?
The NIH budget increased by $2.8 billion during the post-doubling era, only 3% of which was allocated to the R01 program.
by $2.8 billion
by $2.8 billion
by $2.8 billion
by $2.8 billion
by $2.8 billion
by $2.8 billion
by $2.8 billion
by $2.8 billion
1.04 s ± 118 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# Get the answer from all dataset using T5 model
for index, row in df.iterrows():
  df.loc[index, 't5answer'] = get_answer(df.loc[index, 'Question'], df.loc[index, 'Paragraph'])

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


# Fine tune

In [3]:
# library to fine tune a T5 model
!pip install simplet5

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 33.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 kB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 52.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for simplet5: filename=simplet5-0.1.4-py3-none-any.whl size=6873 sha256=80dc36cbc4ceedf7ef1b9b5c938e102664242823be7d5c3b716d7c53893e0260
  Stored in directory: /root/.cache/pip/wheels/b4/7d/af/743765400878438a7593f13f89fdf4004dcde0f2a8e6cb6684
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=91d979960f4b37a4c238d8465f389aa317ccfb4eb82d4da89b3801ff28f5a15a
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built simplet5 sacremoses
  Attempting uninstal

In [5]:
# create the T5 model
from simplet5 import SimpleT5
model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [23]:
# Select rows with non nan values
df = df[pd.isnull(df).any(axis=1)==False]
# create a column to store question and context
def get_format(question, context):
    input_text = "question: {} context: {}".format(question, context)
    return input_text
for index, row in df.iterrows():
  df.loc[index, 'format'] = get_format(df.loc[index, 'Question'], df.loc[index, 'Paragraph'])
df = df.rename(columns={"Answer":"target_text", "format":"source_text"})
dx = df[['source_text', 'target_text']]
print(dx.shape)
dx.head(2)

/tmp/ipykernel_28/3650031400.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[index, 'format'] = get_format(df.loc[index, 'Question'], df.loc[index, 'Paragraph'])


(3343, 2)


,source_text,target_text
0,"question: What percentage of the Total NIH Budget is allocated to NIH-Designated Research-Project Grants (RPGs)? context: Figure 1. Trend in allocation of total NIH budget to NIH-designated Research-Project Grants (RPGs). Total NIH spending on RPGs (P01, P42, PN1, U01, U19, UC1, NIGMS P41, and ...","The percentage of the Total NIH Budget allocated to NIH-Designated Research-Project Grants (RPGs) varies by federal fiscal year (FY), but ranged from 27.0-27.7% between FY 2004-2008."
1,"question: How much of the Total NIH Budget was allocated to R01/R01-Equivalent grants during the post-doubling period? context: Figure 1. Trend in allocation of total NIH budget to NIH-designated Research-Project Grants (RPGs). Total NIH spending on RPGs (P01, P42, PN1, U01, U19, UC1, NIGMS P41...",The percentage of the Total NIH Budget allocated to R01/R01-Equivalent grants during the post-doubling period was approximately 17.2%.


In [24]:
# verify no nan values in the csv file
dx.isna().sum()

source_text    0
target_text    0
dtype: int64

In [25]:
# Split train and test to fine tune our t5
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(dx, test_size=0.2, random_state=42)
train_df.shape, test_df.shape

((2674, 2), (669, 2))

In [26]:
# train the model by 3 epochs
model.train(train_df=train_df,
            eval_df=test_df, 
            source_max_token_len=512, 
            target_max_token_len=64, 
            batch_size=8, 
            max_epochs=3, 
            use_gpu=True
           )

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [32]:
# to download the model
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))
download_file('/kaggle/working/outputs/simplet5-epoch-2-train-loss-0.3626-val-loss-0.5412', 'out')

/kaggle/working/out.zip

In [27]:
# Load the fune tuned model
model.load_model("t5","/kaggle/working/outputs/simplet5-epoch-2-train-loss-0.3626-val-loss-0.5412", use_gpu=True)

In [31]:
idx = 2
print(df.loc[idx, 'Question'])
x = model.predict(df.loc[idx, 'source_text'])
print(x[0])
print("--")
print(df.loc[idx, 'target_text'])

What is the full post-doubling era?
The full post-doubling era (FY2003–2008) is a period in which the NIH budget increased by $2.8 billion.
--
The full post-doubling era is the period between FY2003 and FY2008 where the NIH budget increased by $2.8 billion.
